In [2]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import scipy.stats as stats
import matplotlib.pyplot as plt
from uszipcode import SearchEngine
import folium

df = pd.read_csv("nyc_crashes_202301.csv")

In [3]:
# used data cleaning process from hw3

# replaces the value 0.000000 with missing value
df["LONGITUDE"].replace(0.000000, np.nan, inplace=True)
df["LATITUDE"].replace(0.000000, np.nan, inplace=True)

# create a binary missingness indicator for "ZIP CODE" and "BOROUGH"
# 1 is missing 0 is not missing
df['ZIP CODE Missing'] = df['ZIP CODE'].isna().astype(int)
df['BOROUGH Missing'] = df['BOROUGH'].isna().astype(int)

# create a cross table of the missing patterns of "ZIP CODE" and "BOROUGH"
pd.crosstab(df['ZIP CODE Missing'], df['BOROUGH Missing'])

# create a cross table of the missing patterns of "ZIP CODE" and "BOROUGH"
ct = pd.crosstab(df['ZIP CODE Missing'], df['BOROUGH Missing'])

# used same function from STAT3255 notes
sr = SearchEngine()

def nyczip2burough(zip):
    nzip = int(zip)
    if nzip >= 10001 and nzip <= 10282:
        return "MANHATTAN"
    elif nzip >= 10301 and nzip <= 10314:
        return "STATEN ISLAND"
    elif nzip >= 10451 and nzip <= 10475:
        return "BRONX"
    elif nzip >= 11004 and nzip <= 11109:
        return "QUEENS"
    elif nzip >= 11351 and nzip <= 11697:
        return "QUEENS"
    elif nzip >= 11201 and nzip <= 11256:
        return "BROOKLYN"
    else:
        return np.nan
    
# loop through each row in the dataframe
for index, row in df.iterrows():
    
    # check if either ZIP CODE or BOROUGH is missing
    if pd.isnull(row["ZIP CODE"]) or pd.isnull(row["BOROUGH"]):
        
        # check if both LATITUDE and LONGITUDE are available
        if not pd.isnull(row["LATITUDE"]) and not pd.isnull(row["LONGITUDE"]):
            
            # use reverse geocoding to get the zip code and borough
            result = sr.by_coordinates(row["LATITUDE"], row["LONGITUDE"], radius=1, returns=1)
            
            # check if the result is not empty
            if result:
                
                # get the zip code and borough
                zipcode = result[0].zipcode
                borough = nyczip2burough(zipcode)
                
                # update the missing ZIP CODE and BOROUGH in the dataframe
                df.at[index, "ZIP CODE"] = zipcode
                df.at[index, "BOROUGH"] = borough

# convert lower cases to uppercases
df["CONTRIBUTING FACTOR VEHICLE 1"] = df["CONTRIBUTING FACTOR VEHICLE 1"].str.upper()

In [4]:
df.isnull().sum()

CRASH DATE                          0
CRASH TIME                          0
BOROUGH                           570
ZIP CODE                          565
LATITUDE                          524
LONGITUDE                         524
LOCATION                          448
ON STREET NAME                   1735
CROSS STREET NAME                3489
OFF STREET NAME                  4949
NUMBER OF PERSONS INJURED           0
NUMBER OF PERSONS KILLED            0
NUMBER OF PEDESTRIANS INJURED       0
NUMBER OF PEDESTRIANS KILLED        0
NUMBER OF CYCLIST INJURED           0
NUMBER OF CYCLIST KILLED            0
NUMBER OF MOTORIST INJURED          0
NUMBER OF MOTORIST KILLED           0
CONTRIBUTING FACTOR VEHICLE 1      51
CONTRIBUTING FACTOR VEHICLE 2    1713
CONTRIBUTING FACTOR VEHICLE 3    6062
CONTRIBUTING FACTOR VEHICLE 4    6513
CONTRIBUTING FACTOR VEHICLE 5    6631
COLLISION_ID                        0
VEHICLE TYPE CODE 1               125
VEHICLE TYPE CODE 2              2463
VEHICLE TYPE

In [5]:
# create the hour variable
df["CRASH TIME"] = df["CRASH TIME"].astype(str)
df["hour"] = df["CRASH TIME"].str.split(":").str[0].astype(int)
df["minute"] = df["CRASH TIME"].str.split(":").str[1].astype(int)

df["hour"] = df.apply(lambda x: x["hour"] + 1 if x["minute"] >= 30 else x["hour"], axis=1)
df["hour"] = df["hour"].apply(lambda x: 0 if x == 24 else x)

In [6]:
df["injury"] = df["NUMBER OF PERSONS INJURED"].apply(lambda x: 1 if x >= 1 else 0)

In [7]:
# create a dataframe of zip code information

from uszipcode import SearchEngine
import pandas as pd

sr = SearchEngine()

borough_zipcodes = {
    'Bronx': ['10453', '10457', '10460', '10458', '10467', '10468', '10451', '10452', '10456', '10454', '10455', '10459', '10474'],
    'Brooklyn': ['11212', '11213', '11216', '11233', '11238', '11209', '11214', '11228', '11204', '11218', '11219', '11230', '11234', '11236', '11239', '11223', '11224', '11229', '11235', '11201', '11205', '11215', '11217', '11231', '11203', '11210', '11225', '11226', '11207', '11208', '11211', '11222', '11220', '11232', '11206', '11221', '11237'],
    'Manhattan': ['10026', '10027', '10030', '10037', '10039', '10001', '10011', '10018', '10019', '10020', '10036', '10029', '10035', '10010', '10016', '10017', '10022', '10012', '10013', '10014', '10004', '10005', '10006', '10007', '10038', '10280', '10002', '10003', '10009', '10021', '10028', '10044', '10065', '10075', '10128', '10023', '10024', '10025'],
    'Queens': ['11004', '11101', '11102', '11103', '11104', '11105', '11106', '11351', '11354', '11355', '11356', '11357', '11358', '11359', '11360', '11361', '11362', '11363', '11364', '11365', '11366', '11367', '11412', '11423', '11432', '11433', '11434', '11435', '11436', '11109', '11369', '11370', '11372', '11373', '11377', '11378', '11379', '11385'],
    'Staten Island': ['10302', '10303', '10310', '10306', '10307', '10308', '10309', '10312', '10301', '10304', '10305', '10314']
}

zip_info_list = []

for borough, zipcodes in borough_zipcodes.items():
    for zipcode in zipcodes:
        zip_info = sr.by_zipcode(zipcode).__dict__
        zip_info['borough'] = borough
        zip_info_list.append(zip_info)

df1 = pd.DataFrame(zip_info_list)

df1.to_csv('zip_codes.csv')

In [8]:
zip_code_df = pd.read_csv("zip_codes.csv")
df.dropna(subset=['ZIP CODE'], inplace=True)
df.isnull().sum()

CRASH DATE                          0
CRASH TIME                          0
BOROUGH                             5
ZIP CODE                            0
LATITUDE                          106
LONGITUDE                         106
LOCATION                           58
ON STREET NAME                   1710
CROSS STREET NAME                3128
OFF STREET NAME                  4409
NUMBER OF PERSONS INJURED           0
NUMBER OF PERSONS KILLED            0
NUMBER OF PEDESTRIANS INJURED       0
NUMBER OF PEDESTRIANS KILLED        0
NUMBER OF CYCLIST INJURED           0
NUMBER OF CYCLIST KILLED            0
NUMBER OF MOTORIST INJURED          0
NUMBER OF MOTORIST KILLED           0
CONTRIBUTING FACTOR VEHICLE 1      49
CONTRIBUTING FACTOR VEHICLE 2    1584
CONTRIBUTING FACTOR VEHICLE 3    5576
CONTRIBUTING FACTOR VEHICLE 4    5966
CONTRIBUTING FACTOR VEHICLE 5    6071
COLLISION_ID                        0
VEHICLE TYPE CODE 1               119
VEHICLE TYPE CODE 2              2292
VEHICLE TYPE

In [9]:
# covert type to integer so that they are the same in both dataframes and merge based on zip code
df['ZIP CODE'] = df['ZIP CODE'].astype('int64')
merged_df = pd.merge(df, zip_code_df, left_on='ZIP CODE', right_on='zipcode', how='inner')

In [10]:
merged_df.shape

(4911, 61)

In [11]:
merged_df.drop(['minute', 'CRASH TIME', 'LOCATION', 'ZIP CODE Missing', 'BOROUGH Missing', 'Unnamed: 0', '_sa_instance_state', 'state', 'bounds_east', 'lat', 'bounds_north', 'zipcode_type', 'lng', 'bounds_south', 'zipcode', 'timezone', 'major_city', 'radius_in_miles', 'post_office_city', 'area_code_list', 'common_city_list', 'bounds_west', 'borough'], axis=1, inplace=True)
merged_df.shape

(4911, 38)

In [12]:
# define columns to replace nan values
cols_to_replace_nan = ['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
                       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'VEHICLE TYPE CODE 1', 
                       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']

# replace nan values with "Unspecified"
for col in cols_to_replace_nan:
    merged_df[col] = merged_df[col].fillna("Unspecified")

In [13]:
# define columns with missing values
cols_with_missing = ['land_area_in_sqmi', 'water_area_in_sqmi', 'housing_units', 'occupied_housing_units', 'population',
                     'median_home_value', 'population_density', 'median_household_income']
# drop rows with missing values in specified columns
merged_df = merged_df.dropna(subset=cols_with_missing)

In [14]:
num_unique = merged_df[['BOROUGH', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'county', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
                       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'VEHICLE TYPE CODE 1', 
                       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']].nunique()
print("Number of unique values:", num_unique)

Number of unique values: BOROUGH                             5
ON STREET NAME                   1165
CROSS STREET NAME                1136
OFF STREET NAME                  1333
county                              5
CONTRIBUTING FACTOR VEHICLE 1      47
CONTRIBUTING FACTOR VEHICLE 2      28
CONTRIBUTING FACTOR VEHICLE 3      10
CONTRIBUTING FACTOR VEHICLE 4       5
CONTRIBUTING FACTOR VEHICLE 5       3
VEHICLE TYPE CODE 1                55
VEHICLE TYPE CODE 2                68
VEHICLE TYPE CODE 3                16
VEHICLE TYPE CODE 4                11
VEHICLE TYPE CODE 5                 5
dtype: int64


In [15]:
merged_df.shape

(4899, 38)

In [ ]:
merged_df.to_csv("cleaned_nyc_crashes_202301.csv")